## 14.8 Pigeonhole sort

The algorithms presented so far are **comparison sorts**:
they're based on comparing pairs of items, or their keys.
This section shows an example of a **distribution sort**, which
first distributes the items based on their keys and then collects them in order.
The reason for a different kind of sorting algorithm is efficiency.

### 14.8.1 Comparison sort complexity

[Earlier](../04_Iteration/04_6_lists.ipynb#4.6.2-Creating-lists) we assumed that sorting is linear
in the best case, to check the input is already sorted, and
quadratic in the worst case: comparing all items to each other should be
enough to put each item in its correct place in the sequence.

Insertion sort is linear in the best case and quadratic in the worst case,
as it has to compare each unsorted item to at least one and at most all
previously sorted items. Selection sort is always quadratic
because it compares each unsorted item to all other unsorted items.

Our earlier argument only posited that sorting doesn't need more than
quadratic time, but as we have meanwhile seen, human imagination is able to
come up with worse solutions than needed (like bogosort) and with
better solutions than expected (like merge sort, which is always log-linear).
It turns out that log-linear is the lowest worst-case complexity
for comparison sorts, as I'll show next.

Let's assume the *n* items have unique keys.
Therefore, there's a single ascending permutation. As bogosort did, we can see
sorting as searching for the one permutation that is sorted.

If for example we find out that key(A) < key(B), we can discard all permutations where item A comes after (to the right of) item B,
because those aren't sorted.
For each permutation to be removed from the search space,
because A and B are in the wrong order,
there's exactly one other permutation that is kept in the search space,
because it's equal except that A and B swapped places and
hence are in the correct order. In summary, every comparison gives enough
information to discard half the search space.

After one comparison the search space has *n*! / 2 permutations.
After two comparisons it has *n*!/2/2 = *n*!/2² permutations. In general,
after *c* comparisons, the search space has size *n*!/$2^c$.
The algorithm stops when the search space has one permutation (the sorted one),
i.e. when *n*!/$2^c$ = 1, which is the same as *n*! = $2^c$ or *c* = log *n*!.
It has been proven that log *n*! is about the same as *n* log *n* and so that's
the number of comparisons needed.

### 14.8.2 Algorithm

If we assume more about the sorting keys than just being comparable, then
we can use more efficient sorting algorithms, like **pigeonhole sort**.

The algorithm gets its name from the pigeonholes used in mail sorting.
You can see them at start of the second part of the
[BBC programme](https://learn2.open.ac.uk/mod/oucontent/view.php?id=2554724).
All mail to the same postcode, the sorting key, goes in the same pigeonhole.
We need to know in advance the possible keys to create a pigeonhole for each.

Pigeonhole sort uses a map of keys to items with that key.
This can be done with a lookup table of length *k* and
a key function that returns natural numbers from 0 to *k* – 1.
In the first phase (step&nbsp;3 below)
the algorithm distributes the items according to their keys.
In the second phase (steps 4 and 5)
it collects the items from lowest to highest key.

1. let *pigeonholes* be ()
2. repeat *k* times:
   1. append an empty collection to *pigeonholes*
3. for each *item* in *unsorted*:
   1. add *item* to *pigeonholes*[*key*(*item*)]
4. let *sorted* be ()
5. for each *slot* in *pigeonholes*:
   1. for each *item* in *slot*:
      1. append *item* to *sorted*

### 14.8.3 Complexity

The algorithm only uses constant-time operations.
Step&nbsp;2.1 is done *k* times, steps 3 and 5.1.1 are each executed *n* times
to add all items in unsorted order and retrieve them in sorted order.
The complexity is thus Θ(*n* + *k*).
Usually *k* is either a constant or a multiple of *n*, so the complexity of
pigeonhole sort is linear in the length of the input.

Pigeonhole sort is based on the same idea as selection sort: at each step
append the smallest unsorted item to the sorted items.
The difference is that it uses natural number keys and a lookup table to
select the minimum in constant instead of linear time.
This enables pigeonhole sort to process *n* items in linear time,
while selection sort takes quadratic time.

### 14.8.4 Code and tests

Pigeonhole requires an extra parameter to know how many slots to create.

In [1]:
from typing import Callable


def pigeonhole_sorted(unsorted: list, key: Callable, slots: int) -> list:
    """Return a permutation with keys in non-decreasing order.

    Precondition: for each item in unsorted 0 <= key(item) < slots
    """
    pigeonholes = []
    for slot in range(slots):  # noqa: B007
        pigeonholes.append([])
    for item in unsorted:
        pigeonholes[key(item)].append(item)
    result = []
    for slot in pigeonholes:
        for item in slot:
            result.append(item)
    return result

I can't reuse the [same tests](../14_Sorting/14_1_sort_prep.ipynb#14.1.2-Problem-instances)
as for the previous sorting algorithms
because of the extra `slot` parameter and because the key functions don't return
natural numbers from 0 onwards. I must redefine the key functions and
the test table. I will however use the same problem instances.

In [2]:
from algoesup import test

%run -i ../m269_sorting


def value_nat(card: str) -> int:
    """Return 0 to 12 for value A2...9TJQK respectively.

    Preconditions: as for function 'suit'
    """
    return value(card) - 1  # the value function returns 1 to 13


def suit_nat(card: str) -> int:
    """Return 0 to 3 for suit 'C', 'D', 'H' and 'S' respectively.

    Preconditions: as for function 'suit'
    """
    return {"C": 0, "D": 1, "H": 2, "S": 3}[card[1]]


def card_nat(card: str) -> int:
    """Return 0 to 51 according to the sorted order of the card.

    Cards are sorted first by suit, then by value.
    Preconditions: as for function 'suit'
    """
    return suit_nat(card) * 13 + value_nat(card)


pigeonhole_sorted_tests = [
    # case,       unsorted,          key,    slots, sorted
    ('no cards',   [],               card_nat,  52, []),
    ('1 card',     ['AS'],           card_nat,  52, ['AS']),
    ('same card',  ['6D','6D'],      card_nat,  52, ['6D','6D']),
    ('3 cards',    ['JC','8D','TS'], value_nat, 13, ['8D','TS','JC']),
    ('value up',   UP_DOWN,          value_nat, 13, UP_DOWN),
    ('suit down',  UP_DOWN,          suit_nat,   4, ['KC','QD','3H','AS']),
    ('same value', SAME_VALUE,       card_nat,  52, ['TC','TD','TH','TS']),
]

test(pigeonhole_sorted, pigeonhole_sorted_tests)

Testing pigeonhole_sorted...
Tests finished: 7 passed (100%), 0 failed.


### 14.8.5 Performance

Pigeonhole sort takes linear time on all inputs.
Insertion sort is also linear when the input is sorted.
Which do you expect to be faster for sorted inputs:
insertion sort or pigeonhole sort?

___

Insertion sort uses no extra data structure.
It does no swaps and only one comparison if the item is in its sorted place,
so it should be faster. Let's eat the pudding, so to speak.

In [3]:
for doubling in range(5):
    items = list(range(100 * 2**doubling))
    %timeit -r 5 pigeonhole_sorted(items, identity, len(items))

10.8 μs ± 11.2 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)
21.8 μs ± 39.2 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
44.2 μs ± 83.1 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
92.6 μs ± 293 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
187 μs ± 333 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)


Comparing these values to
[those for insertion sort](../14_Sorting/14_3_insertion_sort.ipynb#14.3.4-Performance),
the latter is indeed slightly faster.

Pigeonhole not only has better complexity than comparison sorts,
it's also simpler: there are no recursive calls, swaps or partitions.
It's thus quite faster than our versions of
[merge sort](../14_Sorting/14_5_merge_sort.ipynb#14.5.3-Code-and-performance) or
[quicksort](../14_Sorting/14_6_quicksort.ipynb#14.6.3-Code-and-performance), which aren't in-place either.

⟵ [Previous section](14_7_quicksort_variants.ipynb) | [Up](14-introduction.ipynb) | [Next section](14_9_summary.ipynb) ⟶